In [1]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder
from category_encoders import OneHotEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.feature_selection import RFE
from tabulate import tabulate
import json
import numpy as np


# Encode

- Read data

In [2]:
jobs = pd.read_csv("../data/clean data all.csv")
jobs.head()

,Salary,Level,Year of Experience,Contract Type,Sex,Programming Language,Language,Location
0,20500000.0,Nhân viên,2 năm,Toàn thời gian,Không yêu cầu,java,tiếng anh,Hà Nội
1,36500000.0,Nhân viên,2 năm,Toàn thời gian,Không yêu cầu,Khác,tiếng anh,Hồ Chí Minh
2,22500000.0,Trưởng/Phó phòng,2 năm,Toàn thời gian,Không yêu cầu,Khác,Không yêu cầu,Hà Nội
3,22500000.0,Trưởng nhóm,3 năm,Toàn thời gian,Không yêu cầu,Khác,Không yêu cầu,Hà Nội
4,30000000.0,Nhân viên,3 năm,Toàn thời gian,Không yêu cầu,php,Không yêu cầu,Hà Nội


- Drop Sex, Contract Type

In [3]:
jobs = jobs.drop(columns = ['Sex', 'Contract Type'])
jobs.head()

,Salary,Level,Year of Experience,Programming Language,Language,Location
0,20500000.0,Nhân viên,2 năm,java,tiếng anh,Hà Nội
1,36500000.0,Nhân viên,2 năm,Khác,tiếng anh,Hồ Chí Minh
2,22500000.0,Trưởng/Phó phòng,2 năm,Khác,Không yêu cầu,Hà Nội
3,22500000.0,Trưởng nhóm,3 năm,Khác,Không yêu cầu,Hà Nội
4,30000000.0,Nhân viên,3 năm,php,Không yêu cầu,Hà Nội


- Categorical encoding for Level variable

In [6]:
jobs['Level'].unique()

array(['Nhân viên', 'Trưởng/Phó phòng', 'Trưởng nhóm', 'Thực tập sinh',
       'Quản lý / Giám sát', 'Giám đốc'], dtype=object)

In [7]:

encoder = OrdinalEncoder(categories=[['Thực tập sinh', 'Nhân viên', 'Trưởng nhóm', 'Trưởng/Phó phòng', 'Quản lý / Giám sát', 'Giám đốc']])
encoded_level = encoder.fit_transform(jobs[['Level']])
jobs['Level'] = encoded_level
jobs.head()

,Salary,Level,Year of Experience,Programming Language,Language,Location
0,20500000.0,1.0,2 năm,java,tiếng anh,Hà Nội
1,36500000.0,1.0,2 năm,Khác,tiếng anh,Hồ Chí Minh
2,22500000.0,3.0,2 năm,Khác,Không yêu cầu,Hà Nội
3,22500000.0,2.0,3 năm,Khác,Không yêu cầu,Hà Nội
4,30000000.0,1.0,3 năm,php,Không yêu cầu,Hà Nội


- Categorical encoding for Year of Experience variable

In [8]:
jobs['Year of Experience'].unique()

array(['2 năm', '3 năm', 'Không yêu cầu kinh nghiệm', '1 năm',
       'Dưới 1 năm', '5 năm', '4 năm', 'Trên 5 năm'], dtype=object)

In [9]:
encoder = OrdinalEncoder(categories=[['Không yêu cầu kinh nghiệm', 'Dưới 1 năm', '1 năm', '2 năm', '3 năm', '4 năm', '5 năm', 'Trên 5 năm']])
encoded_level = encoder.fit_transform(jobs[['Year of Experience']])
jobs['Year of Experience'] = encoded_level
jobs.head()

,Salary,Level,Year of Experience,Programming Language,Language,Location
0,20500000.0,1.0,3.0,java,tiếng anh,Hà Nội
1,36500000.0,1.0,3.0,Khác,tiếng anh,Hồ Chí Minh
2,22500000.0,3.0,3.0,Khác,Không yêu cầu,Hà Nội
3,22500000.0,2.0,4.0,Khác,Không yêu cầu,Hà Nội
4,30000000.0,1.0,4.0,php,Không yêu cầu,Hà Nội


- Categorical encoding for Language variable

In [10]:
one_hot_encoder = OneHotEncoder()
job_pl = one_hot_encoder.fit_transform(jobs['Language'])

jobs = pd.concat([jobs, job_pl], axis=1)
jobs = jobs.drop(columns = ['Language'])
jobs.head()

,Salary,Level,Year of Experience,Programming Language,Location,Language_1,Language_2,Language_3
0,20500000.0,1.0,3.0,java,Hà Nội,1,0,0
1,36500000.0,1.0,3.0,Khác,Hồ Chí Minh,1,0,0
2,22500000.0,3.0,3.0,Khác,Hà Nội,0,1,0
3,22500000.0,2.0,4.0,Khác,Hà Nội,0,1,0
4,30000000.0,1.0,4.0,php,Hà Nội,0,1,0


- Categorical encoding for Location variable

In [11]:
one_hot_encoder = OneHotEncoder()
job_pl = one_hot_encoder.fit_transform(jobs['Location'])

jobs = pd.concat([jobs, job_pl], axis=1)
jobs = jobs.drop(columns = ['Location'])
jobs.head()

,Salary,Level,Year of Experience,Programming Language,Language_1,Language_2,Language_3,Location_1,Location_2,Location_3,Location_4
0,20500000.0,1.0,3.0,java,1,0,0,1,0,0,0
1,36500000.0,1.0,3.0,Khác,1,0,0,0,1,0,0
2,22500000.0,3.0,3.0,Khác,0,1,0,1,0,0,0
3,22500000.0,2.0,4.0,Khác,0,1,0,1,0,0,0
4,30000000.0,1.0,4.0,php,0,1,0,1,0,0,0


- Categorical encoding for Programming Language variable

In [12]:
one_hot_encoder = OneHotEncoder()
job_pl = one_hot_encoder.fit_transform(jobs['Programming Language'])

jobs = pd.concat([jobs, job_pl], axis=1)
jobs = jobs.drop(columns = ['Programming Language'])
jobs

,Salary,Level,Year of Experience,Language_1,Language_2,Language_3,Location_1,Location_2,Location_3,Location_4,...,Programming Language_2,Programming Language_3,Programming Language_4,Programming Language_5,Programming Language_6,Programming Language_7,Programming Language_8,Programming Language_9,Programming Language_10,Programming Language_11
0,20500000.0,1.0,3.0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,36500000.0,1.0,3.0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
2,22500000.0,3.0,3.0,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,22500000.0,2.0,4.0,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,30000000.0,1.0,4.0,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4381,2500000.0,0.0,0.0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4382,12000000.0,1.0,1.0,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4383,14000000.0,1.0,2.0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
4384,15000000.0,2.0,4.0,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0


# Correlation with target variable

In [13]:
import pandas as pd

# Tính toán ma trận tương quan
correlation_matrix = jobs.corr()

# Hiển thị tương quan của từng đặc trưng với biến mục tiêu
target_correlation = correlation_matrix['Salary']
print(target_correlation)


Salary                     1.000000
Level                      0.364446
Year of Experience         0.642902
Language_1                -0.030614
Language_2                 0.003466
Language_3                 0.070498
Location_1                 0.037651
Location_2                 0.019534
Location_3                -0.035216
Location_4                -0.103048
Programming Language_1     0.204407
Programming Language_2    -0.338798
Programming Language_3    -0.013640
Programming Language_4     0.125791
Programming Language_5    -0.017747
Programming Language_6     0.094721
Programming Language_7     0.081276
Programming Language_8     0.143200
Programming Language_9     0.137802
Programming Language_10    0.065903
Programming Language_11    0.010875
Name: Salary, dtype: float64


- List Models

In [14]:
models = {
    'LinearRegression': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'RandomForestRegressor': RandomForestRegressor(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'SVR': SVR(),
    'XGB': xgb.XGBRegressor()
}


In [15]:
X = jobs.drop(columns = ['Salary'])
y = jobs['Salary']

In [16]:
import numpy as np
from sklearn.model_selection import cross_val_score, KFold

# Khởi tạo KFold để kiểm định chéo
kf = KFold(n_splits=5, shuffle=True, random_state=42)

results = []

for name, model in models.items():
    # Đánh giá mô hình sử dụng cross-validation và r2 làm thước đo
    scores = cross_val_score(model, X, y, cv=kf, scoring='r2')
    # Tính R² trung bình
    mean_r2 = np.mean(scores)
    results.append((name, mean_r2))

# Sắp xếp kết quả theo R² Score từ cao xuống thấp
results_df = pd.DataFrame(results, columns=['Model', 'R2 Score']).sort_values(by='R2 Score', ascending=False)

# In kết quả
print(results_df)

# Lấy tên của mô hình có điểm số R² cao nhất
model_best = results_df['Model'].iloc[0]
print("Best performing model:", model_best)


c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.113e+16, tolerance: 3.320e+13
  model = cd_fast.enet_coordinate_descent(
c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.067e+16, tolerance: 3.259e+13
  model = cd_fast.enet_coordinate_descent(
c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the numbe

                       Model  R2 Score
4      RandomForestRegressor  0.628452
7                        XGB  0.616043
5      DecisionTreeRegressor  0.614947
3  GradientBoostingRegressor  0.609686
1                      Ridge  0.522950
2                      Lasso  0.522929
0           LinearRegression  0.522696
6                        SVR -0.019774
Best performing model: RandomForestRegressor


# Feature Selection

In [17]:

# Khởi tạo KFold để kiểm định chéo
kf = KFold(n_splits=5, shuffle=True, random_state=42)

results = []
# Định nghĩa mô hình
model = models[model_best]

# Lặp qua số lượng đặc trưng từ 2 đến số lượng đặc trưng tối đa
for i in range(2, len(X.columns) + 1):
    # Thiết lập RFE với số đặc trưng hiện tại
    selector = RFE(model, n_features_to_select=i, step=1)
    selector.fit(X, y)  # Lưu ý là sử dụng toàn bộ dữ liệu X, y
    
    # Đặc trưng được chọn
    selected_features = X.columns[selector.support_]
    
    # Tạo tập dữ liệu chỉ với các đặc trưng được chọn
    X_selected = selector.transform(X)
    
    # Đánh giá mô hình sử dụng cross-validation
    scores = cross_val_score(model, X_selected, y, cv=kf, scoring='r2')
    
    # Tính R² Score trung bình
    mean_r2 = np.mean(scores)
    
    # Lưu kết quả
    results.append((i, selected_features.tolist(), mean_r2))

# Chuyển kết quả vào DataFrame và sắp xếp theo R²
results_df = pd.DataFrame(results, columns=['Num Features', 'Features', 'R2 Score'])
results_df = results_df.sort_values(by='R2 Score', ascending=False)

# In kết quả
print(results_df)

# Lưu các đặc trưng quan trọng nhất từ mô hình tốt nhất
variable_important = results_df.iloc[0, 1]
print("Most important features:", variable_important)


    Num Features                                           Features  R2 Score
10            12  [Level, Year of Experience, Language_1, Langua...  0.629066
9             11  [Level, Year of Experience, Language_1, Langua...  0.628733
13            15  [Level, Year of Experience, Language_1, Langua...  0.628456
14            16  [Level, Year of Experience, Language_1, Langua...  0.628410
12            14  [Level, Year of Experience, Language_1, Langua...  0.628032
17            19  [Level, Year of Experience, Language_1, Langua...  0.627662
11            13  [Level, Year of Experience, Language_1, Langua...  0.627282
15            17  [Level, Year of Experience, Language_1, Langua...  0.627188
16            18  [Level, Year of Experience, Language_1, Langua...  0.626973
18            20  [Level, Year of Experience, Language_1, Langua...  0.626850
8             10  [Level, Year of Experience, Language_2, Locati...  0.625891
7              9  [Level, Year of Experience, Language_2, Locati

# Training

- List Models and Parameter

In [18]:
# Danh sách các mô hình và tham số tương ứng
models = {
    'RandomForestRegressor': (RandomForestRegressor(), {
        'n_estimators': [100, 200, 500],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt']
    }),
    'XGBRegressor': (xgb.XGBRegressor(), {
        'n_estimators': [100, 200, 500],
        'max_depth': [3, 5, 10],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 1.0],
        'colsample_bytree': [0.8, 1.0],
        'gamma': [0, 0.1, 0.2]
    }),
    'GradientBoostingRegressor': (GradientBoostingRegressor(), {
        'n_estimators': [100, 200, 500],
        'max_depth': [3, 5, 10],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt'],
        'learning_rate': [0.01, 0.1, 0.2]
    }),
    'DecisionTreeRegressor': (DecisionTreeRegressor(), {
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'max_features': ['auto', 'sqrt']
    }),
    'Ridge': (Ridge(), {
        'alpha': [0.1, 1.0, 10.0]
    }),
    'Lasso': (Lasso(), {
        'alpha': [0.1, 1.0, 10.0]
    }),
    'LinearRegression': (LinearRegression(), {}),
    'SVR': (SVR(), {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': [0.1, 1.0, 10.0],
        'epsilon': [0.1, 0.2, 0.5],
        'gamma': ['scale', 'auto']
    })
}

In [19]:

# Phân chia tập dữ liệu
X = jobs[variable_important]
y = jobs['Salary']

# Kết quả lưu tham số tốt nhất
best_params = {}

KFold = KFold(n_splits=5, shuffle=True, random_state=42)

# Đào tạo và điều chỉnh mô hình cho từng loại
for model_name, (model, params) in models.items():
    print(f"Training {model_name}...")
    
    # Thiết lập Grid Search với kiểm định chéo
    grid_search = GridSearchCV(estimator=model, param_grid=params, cv=KFold, scoring='neg_mean_squared_error', verbose=1)
    grid_search.fit(X, y)
    
    # Lưu tham số tốt nhất
    best_params[model_name] = grid_search.best_params_
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")

# Ghi kết quả vào file JSON
with open('best_params_all.json', 'w') as fp:
    json.dump(best_params, fp)

print("All best parameters have been saved to 'best_params.json'.")


Training RandomForestRegressor...
Fitting 5 folds for each of 162 candidates, totalling 810 fits


c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
405 fits failed out of a total of 810.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
405 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\

Best parameters for RandomForestRegressor: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 100}
Training XGBRegressor...
Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Best parameters for XGBRegressor: {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.01, 'max_depth': 10, 'n_estimators': 500, 'subsample': 0.8}
Training GradientBoostingRegressor...
Fitting 5 folds for each of 486 candidates, totalling 2430 fits


c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
1215 fits failed out of a total of 2430.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1215 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\Mai Xuan Truong\AppData\Local\Programs\Pyth

Best parameters for GradientBoostingRegressor: {'learning_rate': 0.01, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 500}
Training DecisionTreeRegressor...
Fitting 5 folds for each of 54 candidates, totalling 270 fits


c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
135 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1467, in wrapper
    estimator._validate_params()
  File "c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\

Best parameters for DecisionTreeRegressor: {'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 5}
Training Ridge...
Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best parameters for Ridge: {'alpha': 1.0}
Training Lasso...
Fitting 5 folds for each of 3 candidates, totalling 15 fits


c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.489e+14, tolerance: 3.256e+13
  model = cd_fast.enet_coordinate_descent(
c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.489e+14, tolerance: 3.256e+13
  model = cd_fast.enet_coordinate_descent(


Best parameters for Lasso: {'alpha': 0.1}
Training LinearRegression...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters for LinearRegression: {}
Training SVR...
Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters for SVR: {'C': 10.0, 'epsilon': 0.5, 'gamma': 'scale', 'kernel': 'poly'}
All best parameters have been saved to 'best_params.json'.


In [20]:
from sklearn.model_selection import KFold
# Đọc file JSON
with open('best_params_all.json', 'r') as fp:
    best_params = json.load(fp)

# Tạo dictionary của các mô hình đã cấu hình
models_configured = {
    'RandomForestRegressor': RandomForestRegressor(**best_params['RandomForestRegressor']),
    'XGBRegressor': xgb.XGBRegressor(**best_params['XGBRegressor']),
    'GradientBoostingRegressor': GradientBoostingRegressor(**best_params['GradientBoostingRegressor']),
    'DecisionTreeRegressor': DecisionTreeRegressor(**best_params['DecisionTreeRegressor']),
    'Ridge': Ridge(**best_params['Ridge']),
    'Lasso': Lasso(**best_params['Lasso']),
    'LinearRegression': LinearRegression(**best_params['LinearRegression']),
    'SVR': SVR(**best_params['SVR'])
}

# Khởi tạo KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Đánh giá mỗi mô hình sử dụng cross-validation và lưu kết quả
results = {}
for name, model in models_configured.items():
    scores = cross_val_score(model, X, y, cv=kf, scoring='r2')
    results[name] = np.mean(scores)

# Sắp xếp kết quả theo R2 Score giảm dần và in ra
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
for model_name, score in sorted_results:
    print(f"{model_name}: R2 Score = {score}")



c:\Users\Mai Xuan Truong\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.489e+14, tolerance: 3.256e+13
  model = cd_fast.enet_coordinate_descent(


GradientBoostingRegressor: R2 Score = 0.634404166864965
RandomForestRegressor: R2 Score = 0.6329226952005712
XGBRegressor: R2 Score = 0.632732700749168
DecisionTreeRegressor: R2 Score = 0.6194176767737389
Ridge: R2 Score = 0.5205246014320157
Lasso: R2 Score = 0.520515265753442
LinearRegression: R2 Score = 0.5205056028305671
SVR: R2 Score = -0.011859279198515704
